# 自动微分

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/beginner/mindspore_autograd.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/master/tutorials/zh_cn/beginner/mindspore_autograd.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/source_zh_cn/beginner/autograd.ipynb)

自动微分能够计算可导函数在某点处的导数值，是反向传播算法的一般化。自动微分主要解决的问题是将一个复杂的数学运算分解为一系列简单的基本运算，该功能对用户屏蔽了大量的求导细节和过程，大大降低了框架的使用门槛。

MindSpore使用`ops.grad`和`ops.value_and_grad`计算一阶导数。`ops.grad`只返回梯度，`ops.value_and_grad`同时返回网络正向计算结果和梯度。`ops.value_and_grad`属性如下：

+ `fn`：待求导的函数或网络。
+ `grad_position`：指定求导输入位置的索引。若为int类型，表示对单个输入求导；若为tuple类型，表示对tuple内索引的位置求导，其中索引从0开始；若是None，表示不对输入求导，这种场景下，`weights`非None。默认值：0。
+ `weights`：训练网络中需要返回梯度的网络变量。一般可通过`weights = net.trainable_params()`获取。默认值：None。
+ `has_aux`：是否返回辅助参数的标志。若为True，`fn`输出数量必须超过一个，其中只有`fn`第一个输出参与求导，其他输出值将直接返回。默认值：False。

本章使用MindSpore中的`ops.value_and_grad`对网络求一阶导数。

## 对网络权重求梯度

由于MindSpore的自动微分建议采用函数式编程，因此样例将以函数式编程方式呈现。

In [1]:
import numpy as np
from mindspore import ops, Tensor
import mindspore.nn as nn
import mindspore as ms

# 定义网络
net = nn.Dense(10, 1)

# 定义损失函数
loss_fn = nn.MSELoss()

# 结合前向网络和损失函数
def forward(inputs, labels):
    logits = net(inputs)
    loss = loss_fn(logits, labels)
    return loss, logits

对权重参数求一阶导，需要将权重传入`ops.value_and_grad`中的`weights`。这里不需要求对输入的导数，将`grad_position`设置成None。

接下来，对网络权重求导：

In [2]:
inputs = Tensor(np.random.randn(16, 10).astype(np.float32))
labels = Tensor(np.random.randn(16, 1).astype(np.float32))
weights = net.trainable_params()

# 这里has_aux设置成True，表示只有loss参与求导，而logits不参与
grad_fn = ops.value_and_grad(forward, grad_position=None, weights=weights, has_aux=True)
(loss, logits), params_gradient = grad_fn(inputs, labels)

# 打印结果
print(logits.shape, len(weights), len(params_gradient))

(16, 1) 2 2


## 停止计算梯度

若某些权重不需要进行求导，则在定义求导网络时，相应的权重参数声明定义的时候，将其属性`requires_grad`需设置为`False`。

In [3]:
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.w = ms.Parameter(ms.Tensor(np.array([6], np.float32)), name='w')
        self.b = ms.Parameter(ms.Tensor(np.array([1.0], np.float32)), name='b', requires_grad=False)

    def construct(self, x):
        out = x * self.w + self.b
        return out

# 构建求导网络
net = Net()
params = net.trainable_params()
x = ms.Tensor([5], dtype=ms.float32)
value, gradient = ops.value_and_grad(net, grad_position=None, weights=params)(x)

print(gradient)

(Tensor(shape=[1], dtype=Float32, value= [ 5.00000000e+00]),)


使用`ops.stop_gradient`可以停止计算梯度，示例如下：

In [4]:
from mindspore.ops import stop_gradient

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.w = ms.Parameter(ms.Tensor(np.array([6], np.float32)), name='w')
        self.b = ms.Parameter(ms.Tensor(np.array([1.0], np.float32)), name='b')

    def construct(self, x):
        out = x * self.w + self.b
        # 停止梯度更新，out对梯度计算无贡献
        out = stop_gradient(out)
        return out

net = Net()
params = net.trainable_params()
x = ms.Tensor([100], dtype=ms.float32)
value, output = ops.value_and_grad(net, grad_position=None, weights=params)(x)

print(f"wgrad: {output[0]}\nbgrad: {output[1]}")

wgrad: [0.]
bgrad: [0.]
